In [1]:
import pandas as pd
from IPython.display import Markdown

import tb_incubator.constants as const
from tb_incubator.constants import set_project_base_path
from tb_incubator.model import build_model
from tb_incubator.plotting import set_plot_label, display_plot, plot_model_vs_actual
from tb_incubator.input import load_targets, load_param_info, get_param_table
from tb_incubator.reporting import load_report_section

pd.options.plotting.backend = "plotly"  
project_paths = set_project_base_path("../tb_incubator/")

## Introduction

In [ ]:
load_report_section('reporting.yaml', 'introduction')
load_report_section('reporting.yaml', 'introduction2')

In [3]:
targets = load_targets()
indicator_names = const.indicator_names
compartments = const.compartments

In [ ]:
#| label: fig-notif
#| fig-cap: "Case notifications for tuberculosis in Indonesia"
#| fig-pos: '!tbph'

notif = targets["notif"]
fig = notif.plot()
fig.update_layout(showlegend=False, xaxis_title='Time')
set_plot_label(fig, indicator_names, "TB case notification")
fig.update_layout(margin={'t':10,'l':10,'b':10,'r':10})
fig.show(renderer="svg", width=500, height=300)

## Model construction

In [5]:
param_info = load_param_info()
params = param_info["value"]

In [ ]:
model, desc = build_model(params)
Markdown(desc)

In [7]:
test_params = {
	'contact_rate': 9.623371230009333,
	'self_recovery_rate': 0.3285404841721671,
	'genexpert_sensitivity': 1.0,
    'progression_multiplier': 1.5778946545692025,
    'base_sensitivity': 0.3,
    #'time_to_screening_end_asymp': 0.1,
    #'screening_scaleup_shape': 0.05,
    #'screening_inflection_time': 1995.3,
    #'seed_time': 1850,
}


In [8]:
model.run(params | test_params)

## Results

In [9]:
outs = model.get_derived_outputs_df()

In [ ]:
load_report_section('reporting.yaml', 'result-compartment')

In [ ]:
#| layout-ncol: 2
#| label: fig-compsize
#| fig-cap: "Base model of TB dynamics in Indonesia."
#| fig-subcap: 
#|   - "Compartment size"
#|   - "Percentage of latent TB infection"
#| fig-pos: '!h'

fig = outs[[f"comp_size_{c}" for c in compartments]].plot.area()
#fig.add_scatter(x=targets["census_pop"].index, y=targets["census_pop"], mode="markers", name="Census data")
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))
set_plot_label(fig, indicator_names, "Compartment size")
display_plot(fig, "comp_size", "svg")

fig = outs["percentage_latent"].plot()
fig.update_layout(showlegend=False)
set_plot_label(fig, indicator_names, "Percentage of latent TB infection (%)")
display_plot(fig, "percentage_latent", "svg")

In [ ]:
load_report_section('reporting.yaml', 'result-notifprev')

In [ ]:
#| label: fig-notifprev
#| fig-cap: "Comparison between model outputs and local data for calibration targets. The red dots represent the actual data from all periods, while the green dots indicate the data specifically used for calibration targets. For prevalence, all actual data is used as the calibration target."
#| fig-subcap: 
#|   - "TB notifications"
#|   - "TB prevalence"
#| layout-ncol: 2
#| fig-pos: '!ht'

fig = plot_model_vs_actual(outs, 
                     targets["notif"],
                     "notification",
                     "Notification",
                     "",
                     "Actual data (All periods)")
#fig.add_scatter(x=targets["new_cases"].index, y=targets["new_cases"], mode="markers", name="NTCP data")
fig.add_scatter(x=targets["notif2000"].index, y=targets["notif2000"], mode="markers", name="Target data")
fig.update_layout(showlegend=False)
fig.update_xaxes(range=[1978, 2023])
display_plot(fig, "notification", "svg")

fig = plot_model_vs_actual(outs, 
                     targets["prevalence"],
                     "prevalence",
                     "Prevalence",
                     "",
                     "Actual data")
fig.update_layout(showlegend=False)
display_plot(fig, "prevalence", "svg")

In [ ]:
load_report_section('reporting.yaml', 'result-conclusion')

In [15]:
#fig = outs[["incidence", "prevalence", "mortality"]].plot()
#set_plot_label(fig, indicator_names, "value")
#display_plot(fig, "inc_prev_mort", "svg")
#fig

In [16]:
#actual_data = targets["notif"] / targets["incidence_num"] * 100
#fig = plot_model_vs_actual(outs, 
                     #actual_data,
                     #"notification_per_incidence",
                     #"Notification per incidence",
                     #"",
                     #"Actual data")

#display_plot(fig, "notification_per_incidence", "svg")

\newpage
## Supplementary

In [ ]:
#| label: tbl-params
#| tbl-cap: Model parameters
#| tbl-cap-location: top

fixed_param_table = get_param_table(param_info)
Markdown(fixed_param_table.to_markdown())


\newpage
## References